# If you want to access the version you have already modified, click "Edit"
# If you want to access the original sample code, click "...", then click "Copy & Edit Notebook"

In [ ]:
## This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        pass
        # print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
_exp_name = "sample"

In [ ]:
# Import necessary packages.
import numpy as np
import torch
import os
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
# "ConcatDataset" and "Subset" are possibly useful when doing semi-supervised learning.
from torch.utils.data import ConcatDataset, DataLoader, Subset, Dataset
from torchvision.datasets import DatasetFolder, VisionDataset

# This is for the progress bar.
from tqdm.auto import tqdm
import random

In [ ]:
myseed = 6666  # set a random seed for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(myseed)
torch.manual_seed(myseed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)

## **Transforms**
Torchvision provides lots of useful utilities for image preprocessing, data wrapping as well as data augmentation.

Please refer to PyTorch official website for details about different transforms.

In [ ]:
# Normally, We don't need augmentations in testing and validation.
# All we need here is to resize the PIL image and transform it into Tensor.
test_tfm = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

# However, it is also possible to use augmentation in the testing phase.
# You may use train_tfm to produce a variety of images and then test using ensemble methods
train_tfm = transforms.Compose([
    # Resize the image into a fixed shape (height = width = 128)
    transforms.Resize((128, 128)),
    # You may add some transforms here.
    
    # 2. 几何变换 (增加数据多样性)
    transforms.RandomHorizontalFlip(p=0.5),        # 水平翻转
    transforms.RandomRotation(15),                 # 随机旋转 +/- 15度
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)), # 随机平移
    
    # 3. 像素变换 (防止过拟合颜色)
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    
    # ToTensor() should be the last one of the transforms.
    transforms.ToTensor(),
])


## **Datasets**
The data is labelled by the name, so we load images and label while calling '__getitem__'

In [ ]:
class FoodDataset(Dataset):

    def __init__(self,path,tfm=test_tfm,files = None):
        super(FoodDataset).__init__()
        self.path = path
        self.files = sorted([os.path.join(path,x) for x in os.listdir(path) if x.endswith(".jpg")])
        if files is not None: #为k折提供的接口
            self.files = files
        print(f"One {path} sample",self.files[0])
        self.transform = tfm
  
    def __len__(self):
        return len(self.files)
  
    def __getitem__(self,idx):
        fname = self.files[idx]
        im = Image.open(fname)
        im = self.transform(im)
        #im = self.data[idx]
        try:
            label = int(fname.split("/")[-1].split("_")[0])
        except:
            label = -1 # test has no label
        return im,label



In [ ]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        # torch.nn.MaxPool2d(kernel_size, stride, padding)
        # input 維度 [3, 128, 128]
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1),  # [64, 128, 128]
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [64, 64, 64]

            nn.Conv2d(64, 128, 3, 1, 1), # [128, 64, 64]
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [128, 32, 32]

            nn.Conv2d(128, 256, 3, 1, 1), # [256, 32, 32]
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [256, 16, 16]

            nn.Conv2d(256, 512, 3, 1, 1), # [512, 16, 16]
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),       # [512, 8, 8]
            
            nn.Conv2d(512, 512, 3, 1, 1), # [512, 8, 8]
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),       # [512, 4, 4]
        )
        self.fc = nn.Sequential(
            nn.Linear(512*4*4, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 11)
        )

    def forward(self, x):
        out = self.cnn(x)
        out = out.view(out.size()[0], -1)
        return self.fc(out)

# [Step 2] 使用 ResNet18 模型
import torchvision.models as models

def get_model():
    # 加载 ResNet18 结构，不使用预训练权重
    model = models.resnet18(pretrained=False)
    
    # 修改全连接层 (fc)，因为 ResNet18 默认输出 1000 类，我们需要 11 类
    # model.fc.in_features 是 ResNet18 最后一层的输入维度 (通常是 512)
    model.fc = nn.Linear(model.fc.in_features, 11) 
    return model

# 验证一下模型是否能跑
# print(get_model())

In [ ]:
_dataset_dir = "../input/ml2022spring-hw3b/food11"

In [ ]:
# [Step 3] 准备 K-Fold 交叉验证
from sklearn.model_selection import KFold
import glob

# 1. 收集所有数据路径 (合并 Training 和 Validation)
# 注意：请确保 _dataset_dir 变量已定义，例如 "../input/ml2022spring-hw3b/food11"
train_dir = os.path.join(_dataset_dir, "training")
val_dir = os.path.join(_dataset_dir, "validation")

# 拿出所有 jpg 文件路径
all_train_files = sorted([os.path.join(train_dir, x) for x in os.listdir(train_dir) if x.endswith(".jpg")])
all_val_files = sorted([os.path.join(val_dir, x) for x in os.listdir(val_dir) if x.endswith(".jpg")])
all_files = np.array(all_train_files + all_val_files) # 合并成一个大池子

# 2. 设置参数
device = "cuda" if torch.cuda.is_available() else "cpu"
n_epochs = 80       # [Train Longer] 建议设为 80-200，取决于你的时间。Simple Baseline只有4。
patience = 15       # Early Stopping 容忍度
batch_size = 64
n_folds = 5         # 5折交叉验证
seed = 6666

# 3. 开始 K-Fold 循环
kf = KFold(n_splits=n_folds, shuffle=True, random_state=seed)

# 为了节省时间，演示代码里你可以只跑一个 Fold，或者把 range(n_folds) 改成 range(1)
# 如果要跑满 Strong Baseline，需要跑完所有 Fold
for fold_idx, (train_idx, val_idx) in enumerate(kf.split(all_files)):
    
    print(f"\n----- Starting Fold {fold_idx + 1} / {n_folds} -----")
    
    # 划分当前 Fold 的文件
    fold_train_files = all_files[train_idx]
    fold_val_files = all_files[val_idx]
    
    # 构建 Dataset 和 DataLoader
    # 注意：我们复用 FoodDataset，利用 files 参数传入文件列表
    train_set = FoodDataset(path=train_dir, tfm=train_tfm, files=fold_train_files)
    valid_set = FoodDataset(path=val_dir, tfm=test_tfm, files=fold_val_files) # 验证集不用增强
    
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
    valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
    
    # 初始化模型 (每个 Fold 都要全新的模型)
    model = get_model().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0003, weight_decay=1e-5)
    
    best_acc = 0
    stale = 0
    
    # 训练循环
    for epoch in range(n_epochs):
        # --- Training ---
        model.train()
        train_loss = []
        train_accs = []
        for batch in tqdm(train_loader, desc=f"Fold {fold_idx+1} Epoch {epoch+1}"):
            imgs, labels = batch
            logits = model(imgs.to(device))
            loss = criterion(logits, labels.to(device))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()
            train_loss.append(loss.item())
            train_accs.append(acc)
            
        avg_train_loss = sum(train_loss) / len(train_loss)
        avg_train_acc = sum(train_accs) / len(train_accs)
        
        # --- Validation ---
        model.eval()
        valid_loss = []
        valid_accs = []
        with torch.no_grad():
            for batch in valid_loader:
                imgs, labels = batch
                logits = model(imgs.to(device))
                loss = criterion(logits, labels.to(device))
                acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()
                valid_loss.append(loss.item())
                valid_accs.append(acc)
                
        avg_valid_loss = sum(valid_loss) / len(valid_loss)
        avg_valid_acc = sum(valid_accs) / len(valid_accs)
        
        print(f"[Fold {fold_idx+1} | {epoch+1}/{n_epochs}] Train Acc: {avg_train_acc:.4f}, Valid Acc: {avg_valid_acc:.4f}")
        
        # 保存最佳模型 (每个 Fold 单独保存)
        if avg_valid_acc > best_acc:
            print(f"New Best Fold {fold_idx+1} Model! Acc: {avg_valid_acc:.4f}")
            torch.save(model.state_dict(), f"{_exp_name}_fold{fold_idx+1}_best.ckpt")
            best_acc = avg_valid_acc
            stale = 0
        else:
            stale += 1
            if stale > patience:
                print(f"Early stopping at epoch {epoch+1}")
                break

## Testing and generate prediction CSV

In [ ]:
# [Step 4] 极简版：集成预测 (Ensemble)
import os
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

# 准备测试集
test_set = FoodDataset(os.path.join(_dataset_dir, "test"), tfm=test_tfm)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)

# 初始化全零数组 [样本数, 11个类别]
final_logits = np.zeros((len(test_set), 11))

# 遍历每一个 Fold 的模型
for fold_idx in range(n_folds):
    model_path = f"{_exp_name}_fold{fold_idx+1}_best.ckpt"
    
    if not os.path.exists(model_path):
        print(f"Model {model_path} not found, skipping.")
        continue
        
    print(f"Inference with {model_path}...")
    
    # 加载模型
    model = get_model().to(device)
    model.load_state_dict(torch.load(model_path))
    model.eval()
    
    fold_preds = []
    with torch.no_grad():
        for data, _ in tqdm(test_loader):
            logits = model(data.to(device))
            fold_preds.append(logits.cpu().numpy())
    
    # 直接累加 Logits
    final_logits += np.concatenate(fold_preds)

# 直接对“和”取最大值索引 (不需要除以模型数量)
prediction = np.argmax(final_logits, axis=1)


In [ ]:
#create test csv
def pad4(i):
    return "0"*(4-len(str(i)))+str(i)
df = pd.DataFrame()
df["Id"] = [pad4(i) for i in range(1,len(test_set)+1)]
df["Category"] = prediction
df.to_csv("submission.csv",index = False)
print("Ensemble submission saved!")